### Iterative testing after laoding model 

In [1]:
import mujoco
import numpy as np
import mujoco.viewer as mv

# Load URDF directly - MuJoCo handles the conversion internally
model = mujoco.MjModel.from_xml_path("/home/blank/Projects/Internship/sentienc/Robotic-Simulation/mujoco-experiments/models/S1/urdf/mjmodel.xml")
print("✅ Model loaded from URDF!")
data = mujoco.MjData(model)

✅ Model loaded from URDF!


In [2]:
# Reset state
mujoco.mj_resetData(model, data)

In [3]:
mujoco.mj_forward(model, data)

In [5]:
print(f"Qvel : {data.qvel}")
print(f"Qacc : {data.qacc}")
print(f"Qpos : {data.qpos}")

Qvel : [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Qacc : [-3.43983136e-04 -1.99027835e-01  1.15488862e+00  1.64219154e+00
  5.80531114e-01  3.93539367e+00  4.83528433e+00 -1.44228240e+00
 -1.63168318e+00 -3.75807237e-01 -4.89510308e+00 -4.22219745e+00
  8.53490658e-01  7.54778729e+01 -9.10700205e+02 -5.78447001e-01
  6.10765135e+00  2.07728751e+03 -1.97684691e+03 -6.04286445e+02
  5.27238461e-01 -3.20458858e+01  1.37386210e+03  1.31209393e+03]
Qpos : [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [6]:
stand_ctrl = np.zeros(model.nu)

# freeze upper body
stand_ctrl[0:14] = 0.0
mujoco.mj_forward(model, data)


In [7]:
print(f"Qvel : {data.qvel}")
print(f"Qacc : {data.qacc}")
print(f"Qpos : {data.qpos}")

Qvel : [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Qacc : [-3.43983136e-04 -1.99027835e-01  1.15488862e+00  1.64219154e+00
  5.80531114e-01  3.93539367e+00  4.83528433e+00 -1.44228240e+00
 -1.63168318e+00 -3.75807237e-01 -4.89510308e+00 -4.22219745e+00
  8.53490658e-01  7.54778729e+01 -9.10700205e+02 -5.78447001e-01
  6.10765135e+00  2.07728751e+03 -1.97684691e+03 -6.04286445e+02
  5.27238461e-01 -3.20458858e+01  1.37386210e+03  1.31209393e+03]
Qpos : [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [8]:
# right leg
stand_ctrl[14] = 0.0
stand_ctrl[15] = -0.3
stand_ctrl[16] = data.qpos[16]   # hip yaw: HOLD current
stand_ctrl[17] = 0.6
stand_ctrl[18] = -0.3

# left leg
stand_ctrl[19] = 0.0
stand_ctrl[20] = -0.3
stand_ctrl[21] = data.qpos[21]
stand_ctrl[22] = 0.6
stand_ctrl[23] = -0.3


In [9]:
print(f"Qvel : {data.qvel}")
print(f"Qacc : {data.qacc}")
print(f"Qpos : {data.qpos}")

Qvel : [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Qacc : [-3.43983136e-04 -1.99027835e-01  1.15488862e+00  1.64219154e+00
  5.80531114e-01  3.93539367e+00  4.83528433e+00 -1.44228240e+00
 -1.63168318e+00 -3.75807237e-01 -4.89510308e+00 -4.22219745e+00
  8.53490658e-01  7.54778729e+01 -9.10700205e+02 -5.78447001e-01
  6.10765135e+00  2.07728751e+03 -1.97684691e+03 -6.04286445e+02
  5.27238461e-01 -3.20458858e+01  1.37386210e+03  1.31209393e+03]
Qpos : [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [12]:
for step in range(2000):
    data.ctrl[:] = stand_ctrl
    mujoco.mj_step(model, data)

    if not np.isfinite(data.qacc).all():
        print("❌ NaN detected at step", step)
        break

    if step % 200 == 0:
        print(f"step {step}, root z = {data.qpos[2]:.3f}")
        print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>.")
        print(f"Qvel : {data.qvel}")
        # print(f"Qacc : {data.qacc}")
        # print(f"Qpos : {data.qpos}")
        print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>.")
        print()


step 0, root z = 2.451
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>.
Qvel : [-3.26790695e-06 -4.57087217e-02 -1.77130239e+00 -8.48301950e+00
  8.99912566e+00  6.77675596e+00  4.80575904e+01 -3.38003676e+00
  4.58962462e-01 -1.97770198e+01  7.93667700e+00 -2.23600337e+01
  3.28448013e-02 -9.82952171e-02  8.78981130e-01 -1.17659534e+00
  1.13211272e-01 -5.97979684e-02  1.42620563e+00 -4.04672390e-02
 -1.21705547e+01 -6.69273121e+00 -5.13141951e+01  8.67731056e+01]
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>.

step 200, root z = 0.127
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>.
Qvel : [-3.26792007e-06 -7.89988512e-02  2.89975446e+00  3.38459856e-01
  7.04982432e+00  3.70501932e+00  1.18839530e+01  3.16698247e+00
 -4.72708549e-01 -6.36370325e+00 -5.53948179e+00 -6.82630721e+00
  3.26894500e-01 -5.97203173e-02 -1.09063494e+00  1.11198662e+00
 -1.07605418e-01  4.49201416e-02 -1.91079117e+00 -1.83570953e-01
 -2.15615015e+01  2.85858807e+01  2.57814770e+01  8.65441751e+01]
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>.

step 400, 